# Executing Matlab code using Octave through python
## Libraries
For executing this mini-tutorial you need to have installed the following libraries:

In [1]:
! conda list octave

# packages in environment at /usr/local/share/miniconda3/envs/env:
#
# Name                    Version                   Build  Channel
octave                    7.3.0                h4bc5dab_3    conda-forge
octave_kernel             0.35.1             pyhd8ed1ab_0    conda-forge


In [2]:
! conda list oct2py

# packages in environment at /usr/local/share/miniconda3/envs/env:
#
# Name                    Version                   Build  Channel
oct2py                    5.6.1              pyhc1e730c_0    conda-forge


In [3]:
import subprocess
import oct2py as ocpy
from oct2py import octave as oc
import os

## Executing matlab/octave functions
Now you can execute matlab/octave as follows

### Option 1: OctPy()

Create an explicit instance of Oct2py given by the module oct2py. This allows more control within the Octave instance and how to interact with it.


In [4]:
# Instantiate octave executor
o2p= ocpy.Oct2Py()
# Run octave basic function
x = o2p.zeros(3, 3)
# Print the result
print(x)
# print result type
print(x.dtype)


[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
float64


### Option 2: octave
Uses a default Octave instanced given by oct2py. This allows to call simple Octave functions in a direct and simpler way without the explicit creation of a Oct2py instance. 

In [5]:
# Run octave basic function
x = ocpy.octave.zeros(3, 3)
# Print the result
print(x)
# print result type
print(x.dtype)


[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
float64


Each of the options has advantages and this advantages:

|                          | Predefined Instance (`oct2py.octave`) | Custom Instance (`oct2py.Oct2Py()`) |
|--------------------------|---------------------------------------|------------------------------------|
| **Advantages**           | - Simplicity                         | - Full control over the instance   |
|                          | - Less code                          | - Ability to have multiple         |
|                          |                                       |   instances                        |
|                          | - Quick usage                        | - Isolation of Octave environment  |
|                          |                                       | - Customization of parameters      |
| **Disadvantages**        | - Less control over the instance     | - More code and complexity         |
|                          |   of Octave                          |                                    |


In concret:

|                                    | Predefined Instance (`oct2py.octave`)                   | Custom Instance (`oct2py.Oct2Py()`)                      |
|------------------------------------|---------------------------------------------------------|----------------------------------------------------------|
| **Allows Configuration**           | No                                                      | Yes                                                      |
| **Allows Path Configuration**      | No                                                      | Yes                                                      |
| **Allows Environment Isolation**   | No (Shares environment with other code)                | Yes (Environment is isolated)                             |
| **Allows Multiple Instances**      | No (Single shared instance)                            | Yes (Multiple instances can be created)                  |
| **Allows Customization**           | Limited                                                 | Extensive                                                |
| **Allows Running Simple Functions**| Yes                                                     | Yes                                                      |
| **Allows Running Complex Scripts** | Yes                                                     | Yes                                                      |
| **Allows Passing Data**            | Yes                                                     | Yes                                                      |
| **Allows Returning Data**          | Yes                                                     | Yes                                                      |


... So ... let's see what happens if I want to execute a .m file

## Execute matlab/octave files from python
### Option 1: Oct2py()
This is the content in the file "hola_mundo.m" so we can check the result is the same

In [6]:
print(o2p.disp('Hola mundo'))




Hola mundo



If we do is as a donkey...

In [7]:
o2p.source('./octave-tests/hola_mundo.m')




Hola Mundo


But we are not donkeys! We are pro 😎

So... let's try to add the path to octave loads path

In [8]:
! ls octave-tests

MP_first_test_penguin_sample.mat  hola_mundo.m	interactiveMatrixProfileVer3.m


In [9]:
current_path = os.getcwd()
path2add = current_path + "/octave-tests/"

print(path2add)
print(current_path)

o2p.addpath(path2add)  # doctest: +SKIP

if os.path.exists(path2add):
    o2p.addpath(path2add)
    print("Path correctly added to octave instance.")
else:
    print("The path is not available")


/home/macu/work/nbs_pipeline/octave-tests/
/home/macu/work/nbs_pipeline






Path correctly added to octave instance.


In [10]:
! ls {path2add}

MP_first_test_penguin_sample.mat  hola_mundo.m	interactiveMatrixProfileVer3.m


In [11]:
listing = o2p.dir(path2add)
myfile = ""
for file in listing:
    print(file.name);
    if file.name == "hola_mundo.m":
        myfile = file




.
..
.ipynb_checkpoints
MP_first_test_penguin_sample.mat
hola_mundo.m
interactiveMatrixProfileVer3.m


In [12]:
o2p.source(myfile.name)

Oct2PyError: Octave evaluation error:
error: source: error sourcing file '/home/macu/work/nbs_pipeline/hola_mundo.m'

Maybe not that pro...

Let's change matlab execution dir

In [ ]:
print(path2add)
print(o2p.cd(path2add))

In [ ]:
o2p.source(myfile.name)

Now yes!! And still python and notebook in the correct place?

In [ ]:
os.getcwd()
! dir

Yes! So... We can setup the octove work path without modyfing python and console paths => We can have our matlab functions in a folder and call them without any hedeaches.

### Option 2: octave
Now let's see if ChatGPT lied to us when saying octove instance couldn't modify the load path.

In [ ]:
print(ocpy.octave.disp('Hola mundo'))
ocpy.octave.source('./octave-tests/hola_mundo.m')

In [ ]:
# Setup the path
current_path = os.getcwd()
path2add = current_path + "/octave-tests/"

print("add:", path2add)
print("current:", current_path)

print("--> Add path")
ocpy.octave.addpath(path2add)  # doctest: +SKIP


In [ ]:
print("--> Show final octave path")
print(ocpy.octave.disp(ocpy.octave.path()))

In [ ]:
print(ocpy.octave.source("hola_mundo.m"))

In [ ]:
print(path2add)
print(ocpy.octave.cd(path2add))
ocpy.octave.source("hola_mundo.m")

## Conclusions
ChatGPT lied to us! 
The load path can also be changued in the default 'octove' instance. 

Thus, the easier way to get the job done is: 

1) Import the libraries

In [ ]:
import subprocess
import os
from oct2py import octave

2) Build and add your .m files path

In [ ]:
current_path = os.getcwd()
path2add = current_path + "/octave-tests/"
print(path2add)

if os.path.exists(path2add):
    octave.addpath(path2add)
    print("Path correctly added to octave instance.")
else:
    print("The path is not available")

3) Move to the path and execute your .m file!

In [ ]:
print(octave.cd(path2add))
octave.source("hola_mundo.m")

4) And don't worry! The rest of the jupyter notebook will keep in its place

In [ ]:
print("---> Python is here")
print(os.getcwd())
print("---> Console too!")
! pwd

## ... And can I exec Eamonn Matrix Profile examples?

In [ ]:
filename = 'interactiveMatrixProfileVer3'
! ls octave-tests/{filename}.m

In [ ]:
import numpy as np
import dvats.memory as mem
import dvats.mplots as mp

In [ ]:
! wget https://www.cs.ucr.edu/~eamonn/MP_first_test_penguin_sample.mat

In [ ]:
filename = 'MP_first_test_penguin_sample.mat'
! mv {filename} octave-tests/{filename}

In [ ]:
ls octave-tests/{filename}

In [ ]:
## -- Classes & types
from dataclasses import dataclass, field
from typing import List
import warnings 

... Necesitamos hacer un equivalente al smooth de matlab.... 

Check... [Matlab: smoothing-data ](https://es.mathworks.com/help/curvefit/smoothing-data.html)
Following [StackOverflow: Matlabs smooth implementation n point moving average in numpy python](https://stackoverflow.com/questions/40443020/matlabs-smooth-implementation-n-point-moving-average-in-numpy-python)

In [ ]:
@dataclass
class MatlabMatrix: 
    filename : str = ""
    matname : str = ""
    data : List [ float ] = None
    data_dict : dict = field (default_factory = dict)
    smoothing_window_len : int = None
    
    def load(self, numcol : int, print_flag : bool = False):
        self.dict = octave.load(self.filename)
        data = self.dict[self.matname]
        if print_flag: print(data.dtype)
        self.data = np.array(data[:,numcol])
        return self.data
    def smooth(self, window_len=11, print_flag = False):
        # window_len: smoothing window size needs, which must be odd number,
        # Step 1: Apply the moving average to the main part of the data using convolution
        self.smoothing_window_len = window_len
        if self.smoothing_window_len % 2 == 0:
            warnings.warn("Window len must be odd! Adding 1 to your length.")
            self.smoothing_window_len += 1
        if print_flag: print("---> About to get out0")
        out0 = np.convolve(
            self.data, 
            np.ones(self.smoothing_window_len, dtype=int),
            'valid'
        ) / self.smoothing_window_len
    
        # Step 2: Handle the beginning of the array (start)
        # Use cumulative sum and then average it by the increasing window size
    
        if print_flag : 
            print("out0 ~", out0.shape)
            print("---> About to get start")
    
        r = np.arange(1, self.smoothing_window_len-1, 2)
    
        if print_flag: print("r", r)
    
        start = np.cumsum(self.data[:self.smoothing_window_len-1])[::2] / r
    
        if print_flag:
            print("start", start)
            print("---> About to get stop")
    
        # Step 3: Handle the end of the array (stop)
        # Reverse the array, use cumulative sum, and then average it by the increasing window size
    
        stop = (np.cumsum(self.data[:-self.smoothing_window_len:-1])[::2] / r)[::-1]

        if print_flag: 
            print("stop", stop)
    
        # Step 4: Combine the start, middle, and end parts together
        return np.concatenate((start, out0, stop))



In [ ]:
penguins_sample = MatlabMatrix(filename = 'MP_first_test_penguin_sample', matname = 'penguin_sample')
penguins_sample.load( numcol = 0, print_flag = True )

In [ ]:
len(penguins_sample.data)

In [ ]:
subsequence_len = 800

In [ ]:
#octave.register_graphics_toolkit('fltk')

In [ ]:
octave.available_graphics_toolkits()

In [ ]:
octave.graphics_toolkit('gnuplot')

In [ ]:
subsequence_len = 800

In [ ]:
len(penguins_sample.data)

In [ ]:
penguins_sample.smooth(window_len = 10, print_flag = True)
[mp, p_index, motif_index, discord_index] = octave.interactiveMatrixProfileVer3(
    penguins_sample.data,
    subsequence_len
);

> Trial following Melvin's tutorial [GitHub](https://melvincabatuan.github.io/Basic-Octave-Tutorial-in-Notebook/)

In [ ]:
%load_ext oct2py.ipython

In [ ]:
%octave ver

In [ ]:
x = %octave [1 2; 3 4];
x

In [ ]:
%octave available_graphics_toolkits()

In [ ]:
%%octave -f svg
t = linspace(-10, 10, 1000)

In [ ]:
%reload_ext oct2py.ipython

In [ ]:
%%octave -f svg
graphics_toolkit('gnuplot');
plot(t,sin(t))